# Manifold Sculpting

In [122]:
import matplotlib.pyplot as plt
import torch
from math import pi

In [76]:
k = 2
data = torch.Tensor([[1,1],[7,7],[9,9],[12,12], [5,5], [2,2]])
p = data[2]
sigma = 0.1
n_dim = 1
scale_factor = 1 # at first

data_size = torch.tensor(data.size())
m = data_size[0] # number of data points
p_size = data_size[1:] # size of the single datapoint
p_size0 = p_size - torch.ones_like(p_size) # to be used for indexing
print(m)
print(p_size)

tensor(6)
tensor([2])


### Compute distances

In [6]:
def neighbor_distance(data):
    x2 = data * data
    x2 = x2.sum(axis=1)
    # print(x2)
    data_t = torch.transpose(data, 0, 1) # pb in dim >i (es foto...), credo funzioni cmq
    xx = data@data_t
    # print(xx)
    # print(x2.unsqueeze(dim=1))
    dist = torch.sqrt( x2 - 2*xx + x2.unsqueeze(dim=1) )
    # print(dist)
    return dist


def avg_neighb_distance(data):
    dist = neighbor_distance(data)
    avg_dist /= data.size()[0] # che sarebbe m
    return avg_dist

dist = neighbor_distance(data)

tensor([[ 0.0000,  8.4853, 11.3137, 15.5563,  5.6569,  1.4142],
        [ 8.4853,  0.0000,  2.8284,  7.0711,  2.8284,  7.0711],
        [11.3137,  2.8284,  0.0000,  4.2426,  5.6569,  9.8995],
        [15.5563,  7.0711,  4.2426,  0.0000,  9.8995, 14.1421],
        [ 5.6569,  2.8284,  5.6569,  9.8995,  0.0000,  4.2426],
        [ 1.4142,  7.0711,  9.8995, 14.1421,  4.2426,  0.0000]])


In [39]:
# extract the neighbours
_, indices = torch.sort(dist)
# print(indices)
# print(dist)
# keep the first k (without the point itself)
kneighb = indices[:, 1:k+1]
kdist   = torch.zeros_like(kneighb)

a = []
for i in range(m):
    # per qlche motivo nn si riesce a farlo direttamente con i torch.tensor
    a.append( dist[kneighb[i][:], i] )
# print(a)
kdist = torch.reshape( torch.cat( a, 0 ), (-1, k) )
del a

# print(kdist)

In [ ]:
# Compute angles
# Loop over points
# Variables with _idx run from 0 to m-1, they select points from `data`
# Variables without _idx run from 0 to k-1

# Entry i,j hosts the info (angle theta, kindex with respect to the neighborhood of j)
# about the most colinear point of the couple i-j ie i-j-l
theta = torch.ones((m,k))
colinear = torch.ones((m,k))
# Loop over data points
for i_idx in range(m):
    # Loop over neighbor points of i
    for j_kidx, j_idx in enumerate(kneighb[i_idx]): # che me ne faccio dl'enumerate
        p2nj = data[i_idx] - data[j_idx]
        p2nj /= torch.norm(p2nj)

        # angle = torch.ones(k)
        colinear_kidx =  torch.ones(k)
        # Loop over neighbor points of j
        for l_kidx, l_indx in enumerate(kneighb[j_idx]):
            p2nl = data[l_idx] - data[j_idx]
            p2nl /= torch.norm(p2nl)

            cos, cos_kidx = torch.sort(p2nl@c2nj, descending=True)
            # extract the collinear angle and neighbor, 0 is the maximum
            colinear[i, j_kidx] = cos_kidx[0] 
            theta[i, j_kidx] = torch.acos(cos[0]).item()
            # colinear_kidx[l_kidx] = cos_kidx[0] 
            # angle[l_kidx] = torch.acos(cos[0]).item()

dist_avg = torch.mean(kdist)

In [50]:
## Test cell
a = torch.Tensor([1,2])
b = torch.Tensor([2,3])
c = a@b
d = c.item()
a.tolist()
a[0]

tensor(1.)

In [65]:
from math import acos
a = torch.Tensor([0,1])
b = torch.Tensor([1,1])
a /= torch.norm(a)
b /= torch.norm(b)
print(b)
c = torch.acos(a@b)
c

tensor([0.7071, 0.7071])


tensor(0.7854)

In [ ]:
# PCA
cov = torch.cov(data)
eigenval, eigenvec = torch.linalg.eigh(-cov) # li ordina già in vista dla pca!
eigenval = -eigenval  # pké li ordina decrescenti
pca_data = eigenvec@data # è qla giusta o è qla trasposta?

preserv_dim = torch.tensor(list(range(n_dim)))
scaled_dim  = torch.tensor(list(range(n_dim, p_size)))

In [75]:
## Test cell
a = torch.Tensor([[1,1,0],
                  [1,4,0],
                  [0,0,3]])
eigenval, eigenvec = torch.linalg.eigh(-a)
eigenval = -eigenval
print(eigenval)
print(eigenvec)

tensor([4.3028, 3.0000, 0.6972])
tensor([[ 0.2898,  0.0000, -0.9571],
        [ 0.9571,  0.0000,  0.2898],
        [ 0.0000,  1.0000,  0.0000]])


In [110]:
a = [1,2]
if [0]:
    print("che tavanata")
while a:
    a.pop(0)
    print("ush")


che tavanata
ush
ush


In [ ]:
## Part 4 (core)
## Untested (ma basta 1 cfr con il codice dl fratello)
while (stopping criterion):
    # Punto a
    # downscale the unwanted dimensions
    scale_factor *= sigma
    pca_data[:, scaled_dim] *= sigma
    
    while (avg_neighb_distance(pca_data) > dist_avg):
        pca_data[:, preserv_dim] /= sigma
        
    # Punto b
    rand_indx = torch.multinomial(torch.ones(m), num_samples=1)
    pr = data[rand_indx,:]
    
    q = []
    q.append(pr)
    visited = []

    steps = 0
    mean_error = 0  # mi sfugge a cosa serva...
    counter = 0 # mi sfugge a cosa serva...
    while q:  # while there are elements in q
        p = q.pop(0)
        if p in visited:
            continue

        # Add p's neighbors in the queue
        for n in kneighb[p]:
            q.append(n)
            
        # ADJUST POINT missing
        # s, err = _adjust_point(p,visited) 

        step += s
        mean_error += err
        counter += 1
        visited.append(p)

    mean_error /= counter

    # numbers from author (1 rozzo weight decay, sembra)
    # if step < m:
    #     learning_rate *= 0.87
    # else:
    #     learning_rate /= 0.91
        

In [121]:
## Test cell
a = torch.Tensor([[1,2,3,4],[5,6,7,8]])
indx = torch.multinomial(a, num_samples=1) # replacement?
print(indx)
print(a[0,indx[0]])
print(a[1,indx[1]])

tensor([[2],
        [3]])
tensor([3.])
tensor([8.])


In [ ]:
def compute_error(p, neighb, colinear, visited):

    w = torch.ones(k)
    for j in range(k):
        # chi è p? è 1 indice o è proprio 1 .? Direi che è 1 indice
        w[j] = 10 if neighb[p,j] in visited else 1

    total_error = 0
    for i in range(k):
        n = neighb[p,i]
        c = colinear[p,i]

        p2n = data[p] - data[n]
        c2n = data[c] - data[n]
        p2n /= torch.norm(p2n)
        c2n /= torch.norm(c2n)
        theta = torch.acos(p2n@c2n) # qua fa dei giochi con min e max...

        cos, cos_kidx = torch.sort(p2nl@c2nj, descending=True)

        err_dist = .5*(p2n - dist[p,i]) / avg_dist
        err_theta = (theta - self.theta0[p,i])/pi   # tnto dovremo far tutto come 1 classe alla fine, qndi piuttosto che inventarmi 1 nome...
        total_err += w[i] * (err_dist*err_dist + err_theta*err_theta)
        
    return total_err

# Programma dla mattina:
- Parti leggendo il pezzo che manca dl paper
- controlla che qlo che hai già scritto funzioni 
    (cfr con il bro per il senso generale, che le funzioni facciano qlo che ti aspetti con dle test cells)
- bisogna finire fit
- scrivere adjust point
- organizzare tutto in 1 classe

## Poi incrociare forte le dita pké funzioni su MNIST